From here onward I eneter my code for calculating trnasformer-XL perplexity

In [1]:
import torch
import numpy as np
import torch.nn as nn

from tqdm import tqdm
from utils.data import preprocess, word2idx, Dictionary
import nltk

nltk.download('punkt')
from nltk import sent_tokenize

from utils.analysis import sent_perplexity

[nltk_data] Downloading package punkt to /home/shaderein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/shaderein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
print(torch.__version__)

1.2.0


In [3]:
# story_file = '/content/drive/MyDrive/PhD Thesis/NLP project: human vs machine/Machine/Can this marriage be saved -  APA Sicence Watch_highlighted_for me.docx'

story_file = './data/articles/On a Rainy Day - by Pat Garcia.docx'

article_file = './data/articles/Can this marriage be saved -  APA Sicence Watch.docx'


# story_text, is_highlight = preprocess(story_file)

story_text, _ = preprocess(story_file)

article_text, _ = preprocess(article_file)

## Interruption Sentences

In [11]:
import nltk
nltk.download('brown')
import numpy as np
from nltk.corpus import brown

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [12]:
seed_num = 1

np.random.seed(seed_num)

index_low_lim = 6
num_target_sentences = 10

num_intrp_sentences = num_target_sentences

low_lim_intrp_len = 4
# least number of tokens in an interruption to avoid things \
# like 1-word titles to be used as interruptions

inds_taget_sentences_story = np.random.randint(index_low_lim, len(story_text),
                                               num_intrp_sentences)
inds_taget_sentences_article = np.random.randint(index_low_lim,
                                                 len(article_text),
                                                 num_intrp_sentences)

intrps_source_science = brown.sents(categories='learned')
intrps_source_fiction = brown.sents(categories='fiction')

num_rand_ind = int(1.5 * num_target_sentences)
# mutiply by 1.5 so that later we can exclude the ones that don't fit the criteria

inds_intrp_sentences_science = np.random.randint(index_low_lim,
                                                 len(intrps_source_science),
                                                 num_rand_ind)
inds_intrp_sentences_fiction = np.random.randint(index_low_lim,
                                                 len(intrps_source_fiction),
                                                 num_rand_ind)

intrp_sentences_science = []
intrp_sentences_fiction = []

for i in inds_intrp_sentences_science:

    sent = ' '.join(intrps_source_science[i])
    sent_tokens = tokenizer.tokenize(sent)

    if len(sent_tokens) >= low_lim_intrp_len:
        intrp_sentences_science.append(sent)

for j in inds_intrp_sentences_fiction:

    sent = ' '.join(intrps_source_fiction[j])
    sent_tokens = tokenizer.tokenize(sent)

    if len(sent_tokens) >= low_lim_intrp_len:
        intrp_sentences_fiction.append(sent)

intrp_sentences_science = intrp_sentences_science[:num_target_sentences]
intrp_sentences_fiction = intrp_sentences_fiction[:num_target_sentences]

In [13]:
target_sent_story_all = []
target_sent_story_plus_context_uninterrupted_all = []
target_sent_story_plus_context_interrupted_science_all = []
target_sent_story_plus_context_interrupted_fiction_all = []

target_sent_article_all = []
target_sent_article_plus_context_uninterrupted_all = []
target_sent_article_plus_context_interrupted_science_all = []
target_sent_article_plus_context_interrupted_fiction_all = []

for counter_target in range(num_target_sentences):

  ## primary text: story ##
  ind_target_story_ = inds_taget_sentences_story[counter_target]

  target_sent_story_ = '<|endoftext|>' + story_text[ind_target_story_]
  target_sent_story_all.append(target_sent_story_)
  
  target_sent_story_plus_context = ''
  target_sent_story_plus_context_intrp_science = ''
  target_sent_story_plus_context_intrp_fiction = ''

  for counter_sent_context_story in range(ind_target_story_):
      target_sent_story_plus_context += story_text[counter_sent_context_story]
      target_sent_story_plus_context_intrp_science += story_text[counter_sent_context_story]
      target_sent_story_plus_context_intrp_fiction += story_text[counter_sent_context_story]

  target_sent_story_plus_context += target_sent_story_
  
  target_sent_story_plus_context_intrp_science += intrp_sentences_science[counter_target]
  target_sent_story_plus_context_intrp_science += target_sent_story_

  target_sent_story_plus_context_intrp_fiction += intrp_sentences_fiction[counter_target]
  target_sent_story_plus_context_intrp_fiction += target_sent_story_

  target_sent_story_plus_context_uninterrupted_all.append(target_sent_story_plus_context)
  target_sent_story_plus_context_interrupted_science_all.append(target_sent_story_plus_context_intrp_science)
  target_sent_story_plus_context_interrupted_fiction_all.append(target_sent_story_plus_context_intrp_fiction)

  ## primary text: article ##
  ind_target_article_ = inds_taget_sentences_article[counter_target]

  target_sent_article_ = '<|endoftext|>' + article_text[ind_target_article_]
  target_sent_article_all.append(target_sent_article_)
  
  target_sent_article_plus_context = ''
  target_sent_article_plus_context_intrp_science = ''
  target_sent_article_plus_context_intrp_fiction = ''

  for counter_sent_context_article in range(ind_target_article_):
      target_sent_article_plus_context += article_text[counter_sent_context_article]
      target_sent_article_plus_context_intrp_science += article_text[counter_sent_context_article]
      target_sent_article_plus_context_intrp_fiction += article_text[counter_sent_context_article]

  target_sent_article_plus_context += target_sent_article_
  
  target_sent_article_plus_context_intrp_science += intrp_sentences_science[counter_target]
  target_sent_article_plus_context_intrp_science += target_sent_article_

  target_sent_article_plus_context_intrp_fiction += intrp_sentences_fiction[counter_target]
  target_sent_article_plus_context_intrp_fiction += target_sent_article_

  target_sent_article_plus_context_uninterrupted_all.append(target_sent_article_plus_context)
  target_sent_article_plus_context_interrupted_science_all.append(target_sent_article_plus_context_intrp_science)
  target_sent_article_plus_context_interrupted_fiction_all.append(target_sent_article_plus_context_intrp_fiction)

In [ ]:
### for Transformer-XL ###

# from transformers import TransfoXLTokenizer, TransfoXLLMHeadModel
from transformers import TransfoXLTokenizer, TransfoXLLMHeadModel

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

tokenizer_XL = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
model_XL = TransfoXLLMHeadModel.from_pretrained('transfo-xl-wt103')
model_XL.to(device)

### If using the GPT-2 tokenizer for the transformer-XL model as well
# from transformers import GPT2Tokenizer
# tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2", device='cuda')

# from transformers import GPT2TokenizerFast
# tokenizer_gpt2 = GPT2TokenizerFast.from_pretrained('gpt2-large', device='cuda')


Downloading:   0%|          | 0.00/9.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/856 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

## Primary text being story

In [ ]:
ppl_target_sent_base_story_all = []
ppls_target_sent_plus_context_story_all_uninterrupted = []
ppls_target_sent_plus_context_story_all_interrupted_science = []
ppls_target_sent_plus_context_story_all_interrupted_fiction = []


context_len_story_all_uninterrupted_ = pd.DataFrame(
    context_len_story_all_uninterrupted)
context_len_story_all_interrupted_science_ = pd.DataFrame(
    context_len_story_all_interrupted_science)
context_len_story_all_interrupted_fiction_ = pd.DataFrame(
    context_len_story_all_interrupted_fiction)

for counter_target in range(num_target_sentences):
# for counter_target in range(3):
ppls_target_sent_plus_context_story_all_uninterrupted_ = pd.DataFrame(
    ppls_target_sent_plus_context_story_all_uninterrupted)
ppls_target_sent_plus_context_story_all_interrupted_science_ = pd.DataFrame(
    ppls_target_sent_plus_context_story_all_interrupted_science)
ppls_target_sent_plus_context_story_all_interrupted_fiction_ = pd.DataFrame(
    ppls_target_sent_plus_context_story_all_interrupted_fiction)
    target_sent_story_ = target_sent_story_all[counter_target]
PPL_results_story_all = pd.concat(
    [
        target_sent_story_all_, intrp_sentences_science_,
        intrp_sentences_fiction_, context_len_story_all_uninterrupted_,
        context_len_story_all_interrupted_science_,
        context_len_story_all_interrupted_fiction_,
        ppl_target_sent_base_story_all_,
        ppls_target_sent_plus_context_story_all_uninterrupted_,
        ppls_target_sent_plus_context_story_all_interrupted_science_,
        ppls_target_sent_plus_context_story_all_interrupted_fiction_
    ],
    keys=[
        'target_sent', 'science_interr_sent', 'fiction_interr_sent',
        'context_len_uninterr', 'context_len_interr_science',
        'context_len_interr_fiction', 'PPL_base', 'PPL_w_context_uninterr',
        'PPL_interr_science', 'PPL_interr_fiction'
    ],
    axis=1)
    target_sent_plus_context_story_interrupted_fiction_ = target_sent_story_plus_context_interrupted_fiction_all[counter_target]

    lls_target_sent_story_ = []
    lls_target_sent_plus_context_story_ = []
    lls_target_sent_plus_context_story_interrupted_science_ = []
    lls_target_sent_plus_context_story_interrupted_fiction_ = []

    input_target_sent_tokenized_story_ = tokenizer_XL(target_sent_story_, return_tensors="pt")
    input_target_sent_plus_context_tokenized_story_ = tokenizer_XL(target_sent_plus_context_story_, return_tensors="pt")
    input_target_sent_plus_context_tokenized_story_interrupted_science_ = tokenizer_XL(target_sent_plus_context_story_interrupted_science_, return_tensors="pt")
    input_target_sent_plus_context_tokenized_story_interrupted_fiction_ = tokenizer_XL(target_sent_plus_context_story_interrupted_fiction_, return_tensors="pt")

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    input_target_sent_tokenized_story_ = input_target_sent_tokenized_story_.to(device)
    input_target_sent_plus_context_tokenized_story_ = input_target_sent_plus_context_tokenized_story_.to(device)
    input_target_sent_plus_context_tokenized_story_interrupted_science_ = input_target_sent_plus_context_tokenized_story_interrupted_science_.to(device)
    input_target_sent_plus_context_tokenized_story_interrupted_fiction_ = input_target_sent_plus_context_tokenized_story_interrupted_fiction_.to(device)


    stride = 1
    
    
    #### base PPL ####
    
    for i in tqdm(range(1, input_target_sent_tokenized_story_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_tokenized_story_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_tokenized_story_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100


        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
#             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_story_.append(log_likelihood)

    len_sent_base = len(lls_target_sent_story_[5:]) # reading from 5th element onward because to exclude lls of tokens from '<|endoftext|>'

    ppl_target_sent_base_story_all.append((torch.exp(torch.stack(lls_target_sent_story_[5:]).sum() / len_sent_base)).item())



    #### Plus context PPL ####

    for i in tqdm(range(1, input_target_sent_plus_context_tokenized_story_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_plus_context_tokenized_story_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_plus_context_tokenized_story_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
#             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_plus_context_story_.append(log_likelihood)

    ppls_target_sent_plus_context_story_all_uninterrupted.append((torch.exp(torch.stack(lls_target_sent_plus_context_story_[-len_sent_base:]).sum() / len_sent_base)).item())
    context_len = end_loc - begin_loc
    context_len_story_all_uninterrupted.append(context_len)
    

    #### Interrupted-science, plus context PPL ####

    for i in tqdm(range(1, input_target_sent_plus_context_tokenized_story_interrupted_science_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_plus_context_tokenized_story_interrupted_science_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_plus_context_tokenized_story_interrupted_science_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
    #             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_plus_context_story_interrupted_science_.append(log_likelihood)

    ppls_target_sent_plus_context_story_all_interrupted_science.append((torch.exp(torch.stack(lls_target_sent_plus_context_story_interrupted_science_[-len_sent_base:]).sum() / len_sent_base)).item())
    context_len = end_loc - begin_loc
    context_len_story_all_interrupted_science.append(context_len)


    #### Interrupted-fiction, plus context PPL ####

    for i in tqdm(range(1, input_target_sent_plus_context_tokenized_story_interrupted_fiction_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_plus_context_tokenized_story_interrupted_fiction_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_plus_context_tokenized_story_interrupted_fiction_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
    #             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_plus_context_story_interrupted_fiction_.append(log_likelihood)

    ppls_target_sent_plus_context_story_all_interrupted_fiction.append((torch.exp(torch.stack(lls_target_sent_plus_context_story_interrupted_fiction_[-len_sent_base:]).sum() / len_sent_base)).item())
    context_len = end_loc - begin_loc
    context_len_story_all_interrupted_fiction.append(context_len)


### Primary text being article

In [ ]:
ppl_target_sent_base_article_all = []
ppls_target_sent_plus_context_article_all_uninterrupted = []
ppls_target_sent_plus_context_article_all_interrupted_science = []
ppls_target_sent_plus_context_article_all_interrupted_fiction = []


context_len_story_all_uninterrupted_ = pd.DataFrame(
    context_len_story_all_uninterrupted)
context_len_story_all_interrupted_science_ = pd.DataFrame(
    context_len_story_all_interrupted_science)
context_len_story_all_interrupted_fiction_ = pd.DataFrame(
    context_len_story_all_interrupted_fiction)

for counter_target in range(num_target_sentences):
# for counter_target in range(1):
ppls_target_sent_plus_context_story_all_uninterrupted_ = pd.DataFrame(
    ppls_target_sent_plus_context_story_all_uninterrupted)
ppls_target_sent_plus_context_story_all_interrupted_science_ = pd.DataFrame(
    ppls_target_sent_plus_context_story_all_interrupted_science)
ppls_target_sent_plus_context_story_all_interrupted_fiction_ = pd.DataFrame(
    ppls_target_sent_plus_context_story_all_interrupted_fiction)
    target_sent_article_ = target_sent_article_all[counter_target]
PPL_results_story_all = pd.concat(
    [
        target_sent_story_all_, intrp_sentences_science_,
        intrp_sentences_fiction_, context_len_story_all_uninterrupted_,
        context_len_story_all_interrupted_science_,
        context_len_story_all_interrupted_fiction_,
        ppl_target_sent_base_story_all_,
        ppls_target_sent_plus_context_story_all_uninterrupted_,
        ppls_target_sent_plus_context_story_all_interrupted_science_,
        ppls_target_sent_plus_context_story_all_interrupted_fiction_
    ],
    keys=[
        'target_sent', 'science_interr_sent', 'fiction_interr_sent',
        'context_len_uninterr', 'context_len_interr_science',
        'context_len_interr_fiction', 'PPL_base', 'PPL_w_context_uninterr',
        'PPL_interr_science', 'PPL_interr_fiction'
    ],
    axis=1)
    target_sent_plus_context_article_interrupted_fiction_ = target_sent_article_plus_context_interrupted_fiction_all[counter_target]

    lls_target_sent_article_ = []
    lls_target_sent_plus_context_article_ = []
    lls_target_sent_plus_context_article_interrupted_science_ = []
    lls_target_sent_plus_context_article_interrupted_fiction_ = []

    input_target_sent_tokenized_article_ = tokenizer_XL(target_sent_article_, return_tensors="pt")
    input_target_sent_plus_context_tokenized_article_ = tokenizer_XL(target_sent_plus_context_article_, return_tensors="pt")
    input_target_sent_plus_context_tokenized_article_interrupted_science_ = tokenizer_XL(target_sent_plus_context_article_interrupted_science_, return_tensors="pt")
    input_target_sent_plus_context_tokenized_article_interrupted_fiction_ = tokenizer_XL(target_sent_plus_context_article_interrupted_fiction_, return_tensors="pt")

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    input_target_sent_tokenized_article_ = input_target_sent_tokenized_article_.to(device)
    input_target_sent_plus_context_tokenized_article_ = input_target_sent_plus_context_tokenized_article_.to(device)
    input_target_sent_plus_context_tokenized_article_interrupted_science_ = input_target_sent_plus_context_tokenized_article_interrupted_science_.to(device)
    input_target_sent_plus_context_tokenized_article_interrupted_fiction_ = input_target_sent_plus_context_tokenized_article_interrupted_fiction_.to(device)


    stride = 1
    
    
    #### base PPL ####
    
    for i in tqdm(range(1, input_target_sent_tokenized_article_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_tokenized_article_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_tokenized_article_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100


        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
#             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_article_.append(log_likelihood)

    len_sent_base = len(lls_target_sent_article_[5:]) # reading from 5th element onward because to exclude lls of tokens from '<|endoftext|>'

    ppl_target_sent_base_article_all.append((torch.exp(torch.stack(lls_target_sent_article_[5:]).sum() / len_sent_base)).item())



    #### Plus context PPL ####

    for i in tqdm(range(1, input_target_sent_plus_context_tokenized_article_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_plus_context_tokenized_article_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_plus_context_tokenized_article_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
#             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_plus_context_article_.append(log_likelihood)

    ppls_target_sent_plus_context_article_all_uninterrupted.append((torch.exp(torch.stack(lls_target_sent_plus_context_article_[-len_sent_base:]).sum() / len_sent_base)).item())
    context_len = end_loc - begin_loc
    context_len_article_all_uninterrupted.append(context_len)
    

    #### Interrupted-science, plus context PPL ####

    for i in tqdm(range(1, input_target_sent_plus_context_tokenized_article_interrupted_science_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_plus_context_tokenized_article_interrupted_science_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_plus_context_tokenized_article_interrupted_science_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
    #             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_plus_context_article_interrupted_science_.append(log_likelihood)

    ppls_target_sent_plus_context_article_all_interrupted_science.append((torch.exp(torch.stack(lls_target_sent_plus_context_article_interrupted_science_[-len_sent_base:]).sum() / len_sent_base)).item())
    context_len = end_loc - begin_loc
    context_len_article_all_interrupted_science.append(context_len)


    #### Interrupted-fiction, plus context PPL ####

    for i in tqdm(range(1, input_target_sent_plus_context_tokenized_article_interrupted_fiction_.input_ids.size(1), stride)):
        begin_loc = max(i + stride - len_fix_win_tokens, 0)
        end_loc = min(i + stride, input_target_sent_plus_context_tokenized_article_interrupted_fiction_.input_ids.size(1))
        # print('end_loc wo C: '+str(end_loc))
        trg_len = end_loc - i    # may be different from stride on last loop
        # trg_len = len(input_target_sent_tokenized)
        input_ids = input_target_sent_plus_context_tokenized_article_interrupted_fiction_.input_ids[:,begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():

            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            outputs = model_XL(input_ids, labels=target_ids)
    #             log_likelihood = outputs[0] * trg_len
            log_likelihood = outputs.losses[0][0]

        lls_target_sent_plus_context_article_interrupted_fiction_.append(log_likelihood)

    ppls_target_sent_plus_context_article_all_interrupted_fiction.append((torch.exp(torch.stack(lls_target_sent_plus_context_article_interrupted_fiction_[-len_sent_base:]).sum() / len_sent_base)).item())
    context_len = end_loc - begin_loc
    context_len_article_all_interrupted_fiction.append(context_len)



In [ ]:
PPL_results_story_all.to_csv('PPL_tXL_results_1sent_RainyDayStory_FixLenWin_' +
                             str(len_fix_win_tokens) + '_seed_' +
                             str(seed_num) + '.csv',
                             index=False)


### Saving result for story

In [ ]:
## saving target sentences and interruption sentneces ##
target_sent_story_all_ = pd.DataFrame(target_sent_story_all)
intrp_sentences_science_ = pd.DataFrame(intrp_sentences_science)
intrp_sentences_fiction_ = pd.DataFrame(intrp_sentences_fiction)

## saving context length ##
context_len_article_all_uninterrupted_ = pd.DataFrame(
    context_len_article_all_uninterrupted)
context_len_article_all_interrupted_science_ = pd.DataFrame(
    context_len_article_all_interrupted_science)
context_len_article_all_interrupted_fiction_ = pd.DataFrame(
    context_len_article_all_interrupted_fiction)

## saving ppl ##
ppl_target_sent_base_article_all_ = pd.DataFrame(
    ppl_target_sent_base_article_all)
ppls_target_sent_plus_context_article_all_uninterrupted_ = pd.DataFrame(
    ppls_target_sent_plus_context_article_all_uninterrupted)
ppls_target_sent_plus_context_article_all_interrupted_science_ = pd.DataFrame(
    ppls_target_sent_plus_context_article_all_interrupted_science)
ppls_target_sent_plus_context_article_all_interrupted_fiction_ = pd.DataFrame(
    ppls_target_sent_plus_context_article_all_interrupted_fiction)

PPL_results_article_all = pd.concat(
    [
        target_sent_article_all_, intrp_sentences_science_,
        intrp_sentences_fiction_, context_len_article_all_uninterrupted_,
        context_len_article_all_interrupted_science_,
        context_len_article_all_interrupted_fiction_,
        ppl_target_sent_base_article_all_,
        ppls_target_sent_plus_context_article_all_uninterrupted_,
        ppls_target_sent_plus_context_article_all_interrupted_science_,
        ppls_target_sent_plus_context_article_all_interrupted_fiction_
    ],
    keys=[
        'target_sent', 'science_interr_sent', 'fiction_interr_sent',
        'context_len_uninterr', 'context_len_interr_science',
        'context_len_interr_fiction', 'PPL_base', 'PPL_w_context_uninterr',
        'PPL_interr_science', 'PPL_interr_fiction'
    ],
    axis=1)


In [ ]:
PPL_results_article_all.to_csv(
    'PPL_tXL_results_1sent_APA_marriage_FixLenWin_' + str(len_fix_win_tokens) +
    '_seed_' + str(seed_num) + '.csv',
    index=False)


### Saving results for article

In [ ]:
## saving target sentences and interruption sentneces ##
target_sent_article_all_ = pd.DataFrame(target_sent_article_all)
intrp_sentences_science_ = pd.DataFrame(intrp_sentences_science)
intrp_sentences_fiction_ = pd.DataFrame(intrp_sentences_fiction)


## saving context length ##
context_len_article_all_uninterrupted_ = pd.DataFrame(context_len_article_all_uninterrupted)
context_len_article_all_interrupted_science_ = pd.DataFrame(context_len_article_all_interrupted_science)
context_len_article_all_interrupted_fiction_ = pd.DataFrame(context_len_article_all_interrupted_fiction)

## saving ppl ##
ppl_target_sent_base_article_all_ = pd.DataFrame(ppl_target_sent_base_article_all)
ppls_target_sent_plus_context_article_all_uninterrupted_ = pd.DataFrame(ppls_target_sent_plus_context_article_all_uninterrupted)
ppls_target_sent_plus_context_article_all_interrupted_science_ = pd.DataFrame(ppls_target_sent_plus_context_article_all_interrupted_science)
ppls_target_sent_plus_context_article_all_interrupted_fiction_ = pd.DataFrame(ppls_target_sent_plus_context_article_all_interrupted_fiction)


PPL_results_article_all = pd.concat([target_sent_article_all_, intrp_sentences_science_, intrp_sentences_fiction_, context_len_article_all_uninterrupted_, context_len_article_all_interrupted_science_, context_len_article_all_interrupted_fiction_, ppl_target_sent_base_article_all_, ppls_target_sent_plus_context_article_all_uninterrupted_, ppls_target_sent_plus_context_article_all_interrupted_science_, ppls_target_sent_plus_context_article_all_interrupted_fiction_], keys = ['target_sent', 'science_interr_sent', 'fiction_interr_sent', 'context_len_uninterr', 'context_len_interr_science', 'context_len_interr_fiction','PPL_base', 'PPL_w_context_uninterr', 'PPL_interr_science', 'PPL_interr_fiction'], axis = 1)

In [ ]:
PPL_results_article_all.to_csv('PPL_tXL_results_1sent_APA_marriage_FixLenWin_'+str(len_fix_win_tokens)+'_seed_'+str(seed_num)+'.csv', index=False)